<a href="https://colab.research.google.com/github/brotheroak/Tensorflow/blob/master/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Google Drive mount


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Load MNIST data

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", reshape=False, one_hot=True)
X_train, y_train = mnist.train.images, mnist.train.labels
X_valid, y_valid = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels

print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_valid)))
print("Test Set:       {} samples".format(len(X_test)))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

#Parameter setting

In [0]:
epochs = 100
learning_rate = 0.00001
batch_size = 64
dropout = 0.8

#Implementation: AlexNet

In [0]:
def conv2d(input, filters, kernel_size, strides=1, activation=tf.nn.relu, padding='SAME', name='conv'):
    with tf.variable_scope(name, reuse=False):
        out = tf.layers.conv2d(input, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                         activation=activation, name=name)
    return out

def dense(input, unit, activation=tf.nn.relu, name='dense'):
    with tf.variable_scope(name, reuse=False):
        out = tf.layers.dense(input, unit, activation=activation, name=name)
    return out

def max_pool(input, k, s, name='pool'):
    out = tf.nn.max_pool(input, ksize=[1, k, k, 1], strides=[1, s, s, 1], padding='SAME', name=name)
    return out

In [0]:
def alexnet(x):
    conv1 = conv2d(x, 48, 11, 4, tf.nn.relu, 'SAME', name='conv1')
    pool1 = max_pool(conv1, 2, 2, name='pool1')

    conv3 = conv2d(pool1, 128, 5, 1, tf.nn.relu, 'SAME', name='conv2')
    pool2 = max_pool(conv3, 2, 2, name='pool2')
    norm1 = tf.nn.lrn(pool2, 5, name='norm1')

    conv3 = conv2d(norm1, 192, 3, 1, tf.nn.relu, 'SAME', name='conv3')
    norm2 = tf.nn.lrn(conv3, 5, name='norm2')

    conv3 = conv2d(norm2, 192, 3, 1, tf.nn.relu, 'SAME', name='conv4')
    norm3 = tf.nn.lrn(conv3, 5, name='norm3')

    conv3 = conv2d(norm3, 128, 3, 1, tf.nn.relu, 'SAME', name='conv5')
    norm4 = tf.nn.lrn(conv3, 5, name='norm4')

    dense1 = tf.reshape(norm4, [-1, tf.size(norm4[0])])
    dense1 = dense(dense1, 1024, name='dense1')
    dense2 = tf.nn.dropout(dense1, keep_prob=dropout, name='drop1')
    dense2 = dense(dense2, 1024, name='dense2')
    dense2 = tf.nn.dropout(dense2, keep_prob=dropout, name='drop2')
    dense3 = dense(dense2, 10, tf.nn.softmax, name='dense3')
    return dense3

In [0]:
import tensorflow as tf

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.int32, [None, 10])
# y_one_hot = tf.squeeze(tf.one_hot(y, 10), axis=1)

keep_prob = tf.placeholder(tf.float32, [])  # dropout (keep probability)
x_resize = tf.image.resize_images(x, [224, 224])

# Construct model
pred = alexnet(x)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


#Training and Testing

In [0]:
import numpy as np

saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False,
                                      gpu_options=tf.GPUOptions(allow_growth=True, visible_device_list='0'))) as sess:
    sess.run(init)
    # Keep training until reach max iterations
    print('Training...')
    for i in range(epochs):
        s = np.random.permutation(len(X_train))
        X_train = X_train[s]
        y_train = y_train[s]
        loss_total, acc_total = 0, 0
        for offset in range(0, len(X_train), batch_size):
            end = offset + batch_size
            batch_xs, batch_ys = X_train[offset:end], y_train[offset:end]
            # Fit training using batch data
            _, acc, loss = sess.run([optimizer, accuracy, cost], feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})

            # Calculate batch accuracy & loss
            acc_total += acc
            loss_total += loss
        acc_total /= (len(X_train) / batch_size)
        loss_total /= (len(X_train) / batch_size)
        print("epoch: " + str(i) + ", Training Loss= " + "{:.4f}".format(
            loss_total) + ", Training Accuracy= " + "{:.4f}".format(acc_total))

    print("Optimization Finished!")
    saver.save(sess, './alexnet')

Training...
epoch: 0, Training Loss= 2.1432, Training Accuracy= 0.3345
epoch: 1, Training Loss= 1.8178, Training Accuracy= 0.6830
epoch: 2, Training Loss= 1.7190, Training Accuracy= 0.7543
epoch: 3, Training Loss= 1.6993, Training Accuracy= 0.7677
epoch: 4, Training Loss= 1.6897, Training Accuracy= 0.7750
epoch: 5, Training Loss= 1.6358, Training Accuracy= 0.8375
epoch: 6, Training Loss= 1.6045, Training Accuracy= 0.8642
epoch: 7, Training Loss= 1.5945, Training Accuracy= 0.8723
epoch: 8, Training Loss= 1.5726, Training Accuracy= 0.8976
epoch: 9, Training Loss= 1.5185, Training Accuracy= 0.9535


KeyboardInterrupt: ignored